In [1]:
import streamlit as st 
from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import gdown

In [2]:
# 필요한 라이브러리 import
from googleapiclient.discovery import build
from google.oauth2 import service_account

# 기본 정보 입력 및 API 열기
creds_file_path = 'aiassistant-423712-eb47cff228a1.json'  # 올바른 서비스 계정 JSON 파일 경로 입력

# 파일에서 자격 증명 불러오기
credentials = service_account.Credentials.from_service_account_file(
    creds_file_path, scopes=['https://www.googleapis.com/auth/drive.readonly', 'https://www.googleapis.com/auth/documents.readonly'])

# 서비스 객체 생성
drive_service = build('drive', 'v3', credentials=credentials)
docs_service = build('docs', 'v1', credentials=credentials)

# 폴더 ID 및 파일 이름 설정
folder_id = "1OrIj8JXzMd-VMH2QkdH8Gm55-VXyyoUh"  # 폴더 ID 입력
file_name = "회의록"  # 다운로드하려는 파일 이름 입력

# 폴더 내의 파일 검색
query = f"'{folder_id}' in parents and name = '{file_name}'"
results = drive_service.files().list(q=query, pageSize=10, fields="files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('해당 이름의 파일을 찾을 수 없습니다.')
else:
    file_id = items[0]['id']
    print(f"파일 ID: {file_id}")

    # Google Docs 파일 내용 가져오기
    document = docs_service.documents().get(documentId=file_id).execute()

    # 문서 내용 추출
    content = document.get('body').get('content')

    # 텍스트 추출 함수
    def extract_text(elements):
        text = ''
        for element in elements:
            if 'paragraph' in element:
                for elem in element.get('paragraph').get('elements'):
                    if 'textRun' in elem:
                        text += elem.get('textRun').get('content')
            elif 'table' in element:
                for row in element.get('table').get('tableRows'):
                    for cell in row.get('tableCells'):
                        text += extract_text(cell.get('content'))
            elif 'tableOfContents' in element:
                text += extract_text(element.get('tableOfContents').get('content'))
        return text

    # 텍스트 내용 출력
    doc_text = extract_text(content)


파일 ID: 1IwTNc1C_Twmot9HofNVs_boknDoUfmI1njXNb6fjSLs


In [3]:
# PDF에서 텍스트를 가져온다
# def get_pdf_text(pdf_path):
#     google_path = 'https://drive.google.com/uc?id='
#     file_id = '1dfTqnaO0zcPdiUjP0490S6UEgFn4hqfE-Jo9We7DDmw/edit?usp=drive_link'
#     output_name = 'test.text'
#     gdown.download(google_path+file_id,output_name,quiet=False)
#     text = ""
#     pdf_reader = PdfReader(pdf_path)
#     for page in pdf_reader.pages:
#         text += page.extract_text()
#     return text

#지정된 조건에 따라 주어진 텍스트를 더 작은 덩어리로 분할
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        separators="\\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

#주어진 텍스트 청크에 대한 임베딩을 생성하고 FAISS를 사용하여 벡터 저장소를 생성
def get_vectorstore(text_chunks):
    embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [6]:
import os
os.environ["OPENAI_API_KEY"] = ""

#주어진 벡터 저장소로 대화 체인을 초기화
def get_conversation_chain(vectorstore):
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_message=True)  #ConversationBufferWindowMemory에 이전 대화 저장
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo-16k-0613'),
        retriever=vectorstore.as_retriever(),
        get_chat_history=lambda h: h,
        memory=memory
    ) #ConversationalRetrievalChain을 통해 langchain 챗봇에 쿼리 전송
    return conversation_chain

In [7]:
# PDF 파일 경로 설정
# pdf_path = "The_Adventures_of_Tom_Sawyer.pdf"
# PDF 텍스트 가져오기
# raw_text = get_pdf_text(pdf_path)
# 텍스트에서 청크 검색
text_chunks = get_text_chunks(doc_text)
# PDF 텍스트 저장을 위해 FAISS 벡터 저장소 만들기
vectorstore = get_vectorstore(text_chunks)
# 대화 체인 만들기
m = get_conversation_chain(vectorstore)

C:\Anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
import speech_recognition as sr
import pyttsx3

# 음성 입력 (STT)
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("질문을 해주세요.")
        audio = recognizer.listen(source)
    try:
        question = recognizer.recognize_google(audio, language="ko-KR")
        print("사용자:", question)
        return question
    except sr.UnknownValueError:
        print("음성을 이해할 수 없습니다.")
        return ""
    except sr.RequestError:
        print("음성 서비스에 접근할 수 없습니다.")
        return ""

# 음성 출력 (TTS)
def speak_response(response):
    engine = pyttsx3.init()  # 기본 드라이버로 초기화
    engine.say(response)
    engine.runAndWait()

# 질문 처리 및 답변 생성
def process_question(question):
    # 질문에 대한 답변 가져오기
    response = m({"question": question})
    if 'answer' in response:
        answer = response['answer']
    else:
        print("올바른 응답 키를 찾을 수 없습니다.")
    return answer

if __name__ == "__main__":
    while True:
        question = recognize_speech()
        if question=="멈춰":
            break;
        if question:
            answer = process_question(question)
            print(answer)
            speak_response(answer)


질문을 해주세요.
음성을 이해할 수 없습니다.
질문을 해주세요.
사용자: 몇 초에 뭐 했는지 하나하나 설명해 줘


C:\Anaconda3\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[00:00] Raphael Allstadt: 안녕하세요 여러분. 빠른 개인 정보 보호 및 보안 세션 미팅에 오신 것을 환영합니다.
[00:06] Carlo Thissen: 안녕하세요, 저는 tl;dv의 CEO인 Carlo Thissen입니다.
[00:11] Allan Bettarel: 소개해 주셔서 감사합니다. 시작해 봅시다!

위의 컨텍스트에 따르면, [00:00]에서 Raphael Allstadt는 참석자들을 환영하고 개인 정보 보호 및 보안 세션 미팅을 시작했습니다. [00:06]에서 Carlo Thissen은 자신의 신분을 소개했고, [00:11]에서 Allan Bettarel은 미팅을 시작하자고 말했습니다. 따라서, [00:00], [00:06], [00:11]에서 각각의 참석자들이 한 행동을 설명해주었습니다.


Exception ignored in: <function BSTR.__del__ at 0x00000217D56B5EE0>
Traceback (most recent call last):
  File "C:\Anaconda3\envs\llm\lib\site-packages\comtypes\__init__.py", line 683, in __del__
    def __del__(self, _free=windll.oleaut32.SysFreeString):
KeyboardInterrupt: 


질문을 해주세요.
사용자: 내가 몇 초에 뭐 했고 몇 초에 뭐 했고 며칠 뭐 했는지 하나하나 다 설명해 달라고 했어
Raphael Allstadt는 0초에 인사를 하였습니다.
질문을 해주세요.
사용자: 멈춰
